<a href="https://colab.research.google.com/github/Tiabet/NLP/blob/master/Kboat_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [1]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [12]:
model_name = "xlm-roberta-base"

tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust `num_labels` based on your task


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Define the paths to your JSONL files
train_file = "/content/drive/MyDrive/data/train_double_v2.jsonl"
val_file = "/content/drive/MyDrive/data/val_double_v2.jsonl"
test_file = "/content/drive/MyDrive/data/test_double_v2.jsonl"

# Load datasets
train_dataset = load_dataset('json', data_files=train_file, split='train')
val_dataset = load_dataset('json', data_files=val_file, split='train')
test_dataset = load_dataset('json', data_files=test_file, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
def preprocess_function(examples):
    input_texts = []
    labels = []

    for message_set in examples['messages']:
        user_message = next(msg['content'] for msg in message_set if msg['role'] == 'user')
        assistant_message = next(msg['content'] for msg in message_set if msg['role'] == 'assistant')

        # Append the user's input as the text to be tokenized
        input_texts.append(user_message)

        # Label as 0 (for failure) or 1 (for success) based on the assistant's response
        if assistant_message == "실패":
            labels.append(0)
        else:
            labels.append(1)

    return {'text': input_texts, 'label': labels}

# Apply the preprocessing function to each dataset (train, val, test)
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/79950 [00:00<?, ? examples/s]

Map:   0%|          | 0/8952 [00:00<?, ? examples/s]

Map:   0%|          | 0/6102 [00:00<?, ? examples/s]

In [13]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Tokenize the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/79950 [00:00<?, ? examples/s]

Map:   0%|          | 0/8952 [00:00<?, ? examples/s]

Map:   0%|          | 0/6102 [00:00<?, ? examples/s]

In [16]:
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format("torch", columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
from transformers import XLMRobertaForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_strategy="steps",
    logging_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.639500,0.640022
1000,0.652900,0.636868
1500,0.655000,0.639798
2000,0.661800,0.636810
2500,0.645300,0.636675
3000,0.644000,0.649206
3500,0.636000,0.649238
4000,0.649400,0.637156


In [ ]:
# Generate predictions
predictions = trainer.predict(tokenized_test)

logits = predictions.predictions  # Raw predictions (logits)
labels = predictions.label_ids  # True labels (if present in the test set)

import numpy as np

# Convert logits to predicted class labels (0 or 1)
predicted_labels = np.argmax(logits, axis=-1)

from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")


In [10]:
from sklearn.metrics import classification_report

# Generate a detailed classification report
report = classification_report(labels, predicted_labels, target_names=['Failure', 'Success'])
print(report)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Step 1: Compute probabilities from logits
probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()

# Step 2: Get the probability of the positive class (label 1)
positive_class_probs = probabilities[:, 1]

# Step 3: Compute ROC curve and AUC score
fpr, tpr, thresholds = roc_curve(labels, positive_class_probs)
roc_auc = auc(fpr, tpr)

# Step 4: Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Dashed diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
